In [5]:
import pandas as pd
import numpy as np
import seaborn as snb 
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold 

In [6]:
df=pd.read_csv("US_Regional_Sales_Data.csv")

In [7]:
df.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
0,SO - 000101,In-Store,WARE-UHY1004,31/12/17,31/5/18,14/6/18,19/6/18,USD,6,15,259,12,5,0.075,"1,001.18","1,963.10"
1,SO - 000102,Online,WARE-NMK1003,31/12/17,31/5/18,22/6/18,2/7/18,USD,14,20,196,27,3,0.075,"3,348.66","3,939.60"
2,SO - 000103,Distributor,WARE-UHY1004,31/12/17,31/5/18,21/6/18,1/7/18,USD,21,16,213,16,1,0.050,781.22,"1,775.50"
3,SO - 000104,Wholesale,WARE-NMK1003,31/12/17,31/5/18,2/6/18,7/6/18,USD,28,48,107,23,8,0.075,"1,464.69","2,324.90"
4,SO - 000105,Distributor,WARE-NMK1003,10/4/18,31/5/18,16/6/18,26/6/18,USD,22,49,111,26,8,0.100,"1,476.14","1,822.40"


In [8]:
df.isnull().sum()

OrderNumber         0
Sales Channel       0
WarehouseCode       0
ProcuredDate        0
OrderDate           0
ShipDate            0
DeliveryDate        0
CurrencyCode        0
_SalesTeamID        0
_CustomerID         0
_StoreID            0
_ProductID          0
Order Quantity      0
Discount Applied    0
Unit Cost           0
Unit Price          0
dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OrderNumber       7991 non-null   object 
 1   Sales Channel     7991 non-null   object 
 2   WarehouseCode     7991 non-null   object 
 3   ProcuredDate      7991 non-null   object 
 4   OrderDate         7991 non-null   object 
 5   ShipDate          7991 non-null   object 
 6   DeliveryDate      7991 non-null   object 
 7   CurrencyCode      7991 non-null   object 
 8   _SalesTeamID      7991 non-null   int64  
 9   _CustomerID       7991 non-null   int64  
 10  _StoreID          7991 non-null   int64  
 11  _ProductID        7991 non-null   int64  
 12  Order Quantity    7991 non-null   int64  
 13  Discount Applied  7991 non-null   float64
 14  Unit Cost         7991 non-null   object 
 15  Unit Price        7991 non-null   object 
dtypes: float64(1), int64(5), object(10)
memory

In [10]:
categorical_col=df.select_dtypes(include=["object","category"]).columns
label_encoder=LabelEncoder()
for col in categorical_col:
    df[col]=label_encoder.fit_transform(df[col])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OrderNumber       7991 non-null   int64  
 1   Sales Channel     7991 non-null   int64  
 2   WarehouseCode     7991 non-null   int64  
 3   ProcuredDate      7991 non-null   int64  
 4   OrderDate         7991 non-null   int64  
 5   ShipDate          7991 non-null   int64  
 6   DeliveryDate      7991 non-null   int64  
 7   CurrencyCode      7991 non-null   int64  
 8   _SalesTeamID      7991 non-null   int64  
 9   _CustomerID       7991 non-null   int64  
 10  _StoreID          7991 non-null   int64  
 11  _ProductID        7991 non-null   int64  
 12  Order Quantity    7991 non-null   int64  
 13  Discount Applied  7991 non-null   float64
 14  Unit Cost         7991 non-null   int64  
 15  Unit Price        7991 non-null   int64  
dtypes: float64(1), int64(15)
memory usage: 999

In [12]:
x=df.drop("Unit Price",axis=1)

In [14]:
x.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost
0,10,1,4,9,750,179,338,0,6,15,259,12,5,0.075,0
1,21,2,2,9,750,467,373,0,14,20,196,27,3,0.075,3081
2,32,0,4,9,750,435,22,0,21,16,213,16,1,0.050,4738
3,43,3,2,9,750,371,892,0,28,48,107,23,8,0.075,787
4,54,0,2,2,750,243,593,0,22,49,111,26,8,0.100,808


In [15]:
df.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
0,10,1,4,9,750,179,338,0,6,15,259,12,5,0.075,0,94
1,21,2,2,9,750,467,373,0,14,20,196,27,3,0.075,3081,373
2,32,0,4,9,750,435,22,0,21,16,213,16,1,0.050,4738,66
3,43,3,2,9,750,371,892,0,28,48,107,23,8,0.075,787,124
4,54,0,2,2,750,243,593,0,22,49,111,26,8,0.100,808,73


In [16]:
y=df["Unit Price"]

In [17]:
y.head()

0     94
1    373
2     66
3    124
4     73
Name: Unit Price, dtype: int64

In [18]:
x_train, x_temp, y_train , y_temp=train_test_split(x,y,test_size=0.3,random_state=42)

In [19]:
x_val,x_test,y_val,y_test=train_test_split(x_temp,y_temp,train_size=0.5,random_state=42)

In [20]:
len(x_train)

5593

In [21]:
len(x_val)

1199

In [25]:
model=RandomForestRegressor()

In [26]:
model.fit(x_train,y_train)

RandomForestRegressor()

In [27]:
y_pred=model.predict(x_test)

In [31]:
y_pred

array([114.96, 264.37, 302.1 , ..., 161.1 ,  82.32, 418.79])

In [32]:
len(y_pred)

1199

In [33]:
mse=mean_squared_error(y_test,y_pred)
r2=r2_score(y_test,y_pred)

In [34]:
print(mse)
print(r2)

31435.321100000005
0.32160520608385934


In [35]:
scaler_x=StandardScaler()
scaler_y=StandardScaler()

In [36]:
x_train_scaled=scaler_x.fit_transform(x_train)
x_test_scaled=scaler_x.transform(x_test)

In [37]:
y_train_scaled=scaler_y.fit_transform(y_train.values.reshape(-1,1))
y_test_scaled=scaler_y.transform(y_test.values.reshape(-1,1))

In [38]:
model=RandomForestRegressor()

In [40]:
model.fit(x_train_scaled,y_train_scaled)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor()

In [41]:
y_pred_scaled=model.predict(x_test_scaled)

In [42]:
mse=mean_squared_error(y_test_scaled,y_pred_scaled)
r2=r2_score(y_test_scaled,y_pred_scaled)

In [43]:
print(mse)
print(r2)

0.7083567504465746
0.3236138427386369


In [45]:
scaler_x=StandardScaler()

In [46]:
x.head()

,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost
0,10,1,4,9,750,179,338,0,6,15,259,12,5,0.075,0
1,21,2,2,9,750,467,373,0,14,20,196,27,3,0.075,3081
2,32,0,4,9,750,435,22,0,21,16,213,16,1,0.050,4738
3,43,3,2,9,750,371,892,0,28,48,107,23,8,0.075,787
4,54,0,2,2,750,243,593,0,22,49,111,26,8,0.100,808


In [47]:
kf=KFold(n_splits=5,shuffle=True,random_state=42)

In [48]:
x_scaled=scaler_x.fit_transform(x)

In [49]:
mse=cross_val_score(model,x_scaled,y,cv=kf)

In [50]:
print(mse)

[0.31687763 0.33811121 0.34235128 0.33055722 0.34205035]


In [51]:
avg_mse=np.mean(mse)

In [52]:
avg_mse

np.float64(0.3339895406656158)